In [2]:
!git clone --branch dev https://github.com/rolysr/medical-knowledge-discoverer


Cloning into 'medical-knowledge-discoverer'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 550 (delta 96), reused 157 (delta 60), pack-reused 339
Receiving objects: 100% (550/550), 2.37 MiB | 19.86 MiB/s, done.
Resolving deltas: 100% (243/243), done.


In [3]:
%cd medical-knowledge-discoverer/
!ls

/content/medical-knowledge-discoverer
datasets  main.py  ontology   requirements.txt
LICENSE   models   README.md  utils


In [4]:
!pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 113.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 122.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of es-core-news-sm to determine which version is compatible with other requirements. This co

In [5]:
%%capture
pip install transformers

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

from pathlib import Path
from utils.anntools import Collection
import csv

import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

## Leer Data y crear Dataframe

In [7]:
train_collection = Collection()
for fname in Path("datasets/train").rglob("*.txt"):
    train_collection.load(fname)

test_collection = Collection()
test_collection.load(Path("datasets/test/scenario2-taskA/output.txt"))
#for fname in Path("datasets/test/scenario2-taskA").rglob("*.txt"):
#    test_collection.load(fname)
print(test_collection.sentences)
def build_dataframe(collection):
    lb = []
    txt = []
    for sentence in collection.sentences:
        new_sentence = sentence.text.replace('(','').replace(')','')
        txt.append(new_sentence)
        labels = ['O' for i in range(len(new_sentence.split(' ')))]
        keyphrases = sentence.keyphrases
        sentence_list = new_sentence.split(' ')
        count = 0
        k = 0
        for keyphrase in sentence.keyphrases:
            index = new_sentence.find(keyphrase.text)   
            while count + k < index:
                count += len(sentence_list[k])
                k+=1
            keyphrases_list = keyphrase.text.split(' ')
            if k < len(sentence_list) and sentence_list[k] == keyphrases_list[0]:
                labels[k]= 'B-' + keyphrase.label 
                count += len(keyphrases_list[0])
                i = 1
                while i < len(keyphrases_list):
                    labels[k + i] = 'I-' + keyphrase.label 
                    count += len(keyphrases_list[i])
                    i += 1
                k +=i
        lb.append(labels.copy())

    labels = []

    for label_list in lb:
      current_label = ""
      for label in label_list:
        current_label += label + ' '
      labels.append(current_label)

    data = {
        "text":txt,
        "labels":labels
    }


    df = pd.DataFrame(data)
    
    return df

df = build_dataframe(train_collection)
df_test = build_dataframe(test_collection) # esto hay que arreglarlo para que tenga los labels
df_test.head()

[Sentence(text='Los glóbulos blancos ayudan a su organismo a combatir infecciones.', keyphrases=[Keyphrase(text='glóbulos blancos', label='Concept', id=1, attr=[]), Keyphrase(text='ayudan', label='Action', id=2, attr=[]), Keyphrase(text='organismo', label='Concept', id=3, attr=[]), Keyphrase(text='combatir', label='Action', id=4, attr=[]), Keyphrase(text='infecciones', label='Concept', id=5, attr=[])], relations=[]), Sentence(text='Se contagia a través de relaciones sexuales con una persona infectada.', keyphrases=[Keyphrase(text='Se', label='Reference', id=6, attr=[]), Keyphrase(text='contagia', label='Action', id=7, attr=[]), Keyphrase(text='relaciones sexuales', label='Concept', id=8, attr=[]), Keyphrase(text='persona', label='Concept', id=9, attr=[]), Keyphrase(text='infectada', label='Concept', id=10, attr=[])], relations=[]), Sentence(text='Las lágrimas son producidas por las glándulas lagrimales y el conducto lagrimal lleva estas lágrimas a los ojos.', keyphrases=[Keyphrase(text

,text,labels
0,Los glóbulos blancos ayudan a su organismo a c...,O B-Concept I-Concept B-Action O O B-Concept O...
1,Se contagia a través de relaciones sexuales co...,B-Reference B-Action O O O B-Concept I-Concept...
2,Las lágrimas son producidas por las glándulas ...,O B-Concept O B-Action O O B-Concept I-Concept...
3,"Con frecuencia, las adolescentes no reciben cu...",O O O B-Concept O B-Action B-Concept B-Concept...
4,Phage-mediated horizontal gene transfer events...,B-Concept B-Concept B-Concept B-Action B-Predi...


## Inicializar Tokenizer


In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

## Crear clase Dataset

In [10]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        txt = df['text'].values.tolist()
        
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

        
    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels



## Separar data y definir etiquetas únicas 

In [11]:
unique_labels = {'B-Action', 'B-Concept', 'B-Predicate', 'B-Reference', 'I-Action', 'I-Concept', 'I-Predicate',  'I-Reference', 'O'}

labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

df_train, df_val = np.split(df.sample(frac=1, random_state=42),
                            [int(.7 * len(df))])



## Construir modelo

In [12]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

## Entrenando Modelo

In [1]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 5e-3
EPOCHS = 2
BATCH_SIZE = 2

model = BertModel()
train_loop(model, df_train, df_val)

NameError: ignored

## Predecir una oración

In [ ]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)
            
    return prediction_label

Bill Gates is the founder of Microsoft
['O', 'O', 'O', 'O', 'O', 'O', 'O']


## Evaluando modelo

In [ ]:
def eval_model(df, model):

  CORRECT, INCORRECT, MISSING, SPURIOUS, PARTIAL = 0, 0, 0, 0, 0
  counter = 0
  sentences = list(df['text'])
  print(sentences)
  labels = list(df['labels'])
  print(labels)
  
  for sentence in sentences:
    print(f"sentence {sentence}")
    label = labels[counter].split()
    print(f"label {label}")
    counter += 1
    print(f"Evaluating sentence {counter}/{len(df)}")

    predicted_label = evaluate_one_text(model, sentence)  
    
    print(f"predicted label {predicted_label}")

    for i in range(min(len(label),len(predicted_label))):
      if label[i] == predicted_label[i]:
        CORRECT += 1
        print(f"CORRECT")
      elif label[i] != 'O' and predicted_label[i] != 'O' and label[i] != predicted_label[i]:
        INCORRECT +=1
        print(f"INCORRECT")
      elif label[i] != 'O' and predicted_label[i] == 'O':
        MISSING += 1
        print(f"MISSING")
      elif label[i] == 'O' and predicted_label[i] != 'O':
        SPURIOUS += 1
        print(f"SPURIOUS")
      elif label[i].startswith('B') and predicted_label[i] != 'O' and i < min(len(label),len(predicted_label))-2 and label[i+1].startswith('I'):
        k = i
        while k < min(len(label),len(predicted_label)) or label[k] != 'O':
          if label[k] == predicted_label[k]:
            PARTIAL += 1


    
  rec_b = (CORRECT + 1/2*PARTIAL)/(CORRECT + MISSING + INCORRECT + PARTIAL)
  prec_b =  (CORRECT + 1/2*PARTIAL)/(CORRECT + SPURIOUS + INCORRECT + PARTIAL)
  f1_b = (prec_b*rec_b)/(prec_b+rec_b)

  return rec_b, prec_b, f1_b


rec_b, prec_b, f1_b = eval_model(df_test, model)
print("REC_B: ", rec_b)
print("PREC_B: ", prec_b)
print("F1_B: ", f1_b)


['Some 1,27,000 people are known dead .', "The U.S. rap star Snoop Dogg and five of his associates have been arrested in Britain after a disturbance at London 's Heathrow Airport .", "The global financial crisis has left Iceland 's economy in shambles .", "North Korea says flooding caused by last week 's typhoon , Wipha , has destroyed 14,000 homes and 1,09,000 hectares of crops .", 'An earthquake in late March killed more than 900 people on Nias .']
['O O O O O O O', 'O B-geo O O B-per I-per O O O O O O O O O B-geo O O O O B-geo O B-geo I-geo O', 'O O O O O O B-org O O O O O', 'B-geo I-geo O O O O O O O O O B-per O O O O O O O O O O O', 'O O O O B-tim O O O O O O B-org O']
sentence Some 1,27,000 people are known dead .
label ['O', 'O', 'O', 'O', 'O', 'O', 'O']
Evaluating sentence 1
predicted label ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
sentence The U.S. rap star Snoop Dogg and five of his associates have been arr